#### libics: drivers

# Vialux V-7000 / Texas Instruments DLP7000

Product homepage: [Vialux V-7000](https://www.vialux.de/de/hi-speed-v-module.html)

### Imports

In [ ]:
import os
import time

import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
import PIL

from libics import env
from libics.drv import drv
from libics.drv.itf import itf
from libics.data import types, dataset, stp
from libics.display import plot, plotdefault
from libics.file import hdf
from libics.util import misc

### Configuration

In [ ]:
itf_cfg = {
    "protocol": itf.ITF_PROTOCOL.BINARY,
    "interface": itf.ITF_BIN.VIALUX,
    "device": None
}
itf_cfg = itf.ProtocolCfgBase(**itf_cfg).get_hl_cfg()

In [ ]:
drv_cfg = {
    "driver": drv.DRV_DRIVER.DSP,
    "interface": itf_cfg,
    "identifier": "dmd_vialux_v7000_texasinstruments_dlp7000",
    "model": drv.DRV_MODEL.TEXASINSTRUMENTS_DLP7000,
    "pixel_hrzt_count": 1024,
    "pixel_hrzt_size": 13.68e-6,
    "pixel_hrzt_offset": 0,
    "pixel_vert_count": 768,
    "pixel_vert_size": 13.68e-6,
    "pixel_vert_offset": 0,
    "format_color": drv.DRV_DSP.FORMAT_COLOR.BW,
    "channel_bitdepth": 1,
    "picture_time": 9,
    "dark_time": 0,
    "sequence_repetitions": 0,
    "temperature": 25.0
}
drv_cfg = drv.DrvCfgBase(**drv_cfg).get_hl_cfg()

### Setup

##### Driver

In [ ]:
dsp = drv.DrvBase(cfg=drv_cfg).get_drv()

In [ ]:
def setup(dsp):
    dsp.setup()
    dsp.connect()

def shutdown(dsp):
    dsp.stop()
    dsp.close()
    dsp.shutdown()

def set_pattern(dsp, pattern):
    dsp.stop()
    dsp.init(pattern)
    dsp.write_all()
    dsp.run()

##### Pattern Generator

In [ ]:
def pattern_uniform(val=1, shape=(1024, 768), plot=False):
    pattern = np.full(shape, val, dtype=float)
    if plot:
        plot_pattern(pattern)
    return pattern

def pattern_box(center=(511, 383), size=(1, 1), shape=(1024, 768), plot=False):
    if np.isscalar(size):
        size = (size, size)
    if np.isscalar(center):
        center = (center, center)
    dx = (size[0] - 1) // 2, size[0] - (size[0] - 1) // 2
    dy = (size[1] - 1) // 2, size[1] - (size[1] - 1) // 2
    pattern = np.zeros(shape, dtype=float)
    pattern[
        center[0]-dx[0]:center[0]+dx[1], center[1]-dy[0]:center[1]+dy[1]
    ] = 1
    if plot:
        plot_pattern(pattern)
    return pattern

def pattern_grid(offset=(0, 0), size=(1, 1), shape=(1024, 768), plot=False):
    """
    offset : tuple(int)
        Box pattern (x, y) phase shift.
        0 corresponds to black block, 1 corresponds to white block.
    """
    if np.isscalar(size):
        size = (size, size)
    if np.isscalar(offset):
        offset = (offset, offset)
    block_x, block_y = (
        np.roll(np.concatenate((np.zeros(size[0]), np.ones(size[0]))), offset[0]),
        np.roll(np.concatenate((np.zeros(size[1]), np.ones(size[1]))), offset[1])
    )
    block = np.logical_xor.outer(block_x, block_y)
    repetition = np.ones(np.ceil(np.array(shape) / np.array(size)).astype(int))
    pattern = np.kron(repetition, block)[:shape[0], :shape[1]].astype(float)
    if plot:
        plot_pattern(pattern)
    return pattern

def pattern_ellipse(center=(511, 383), radius=(200, 200), shape=(1024, 768), plot=False):
    if np.isscalar(center):
        center = (center, center)
    if np.isscalar(radius):
        radius = (radius, radius)
    pattern = np.zeros(shape, dtype=float)
    if radius[0] > 0 and radius[1] > 0:
        x, y = np.indices(pattern.shape, dtype=float)
        mask = ((x - center[0])**2 / radius[0]**2 + (y - center[1])**2 / radius[1]**2 <= 1)
        pattern[mask] = 1
    if plot:
        plot_pattern(pattern)
    return pattern

def pattern_line(center=383, size=1, orientation="x", shape=(1024, 768), plot=False):
    """
    center : int
        Center of line.
    size : int
        Width of line.
    orientation : str
        "x" ("horizontal"), "y" ("vertical").
    """
    pattern = np.zeros(shape, dtype=float)
    if orientation == "horizontal" or orientation == "x":
        pattern[center] = 1
    elif orientation == "vertical" or orientation == "y":
        pattern[:, center] = 1
    if plot:
        plot_pattern(pattern)
    return pattern

def plot_pattern(pattern):
    plt.pcolormesh(1 - pattern.T, vmin=0, vmax=1, cmap="Greys")
    plt.gca().set_aspect(1)
    
def save_pattern(pattern, file_path):
    im = PIL.Image.fromarray((pattern * 255).astype("uint8").T)
    file_path = misc.assume_endswith(file_path, ".bmp")
    im.save(file_path)

### Script

In [ ]:
setup(dsp)

In [ ]:
pattern = pattern_uniform(val=1, plot=True)
save_pattern(pattern, os.path.join(
    env.DIR_USER, "Desktop", "pattern_white.bmp"
))
pattern = pattern_grid(size=16, plot=True)
save_pattern(pattern, os.path.join(
    env.DIR_USER, "Desktop", "pattern_grid.bmp"
))

In [ ]:
pattern = pattern_box(size=100, plot=True)
set_pattern(dsp, pattern)

In [ ]:
sizes = np.arange(10)**2
for size in sizes.tolist():
    pattern = pattern_box(size=size, plot=False)
    set_pattern(dsp, pattern)
    print("\rBox dimensions: {0:d}x{0:d}".format(size), end="       ")
    time.sleep(1)

In [ ]:
sizes = np.arange(10)**2 / 2
for size in sizes.tolist():
    pattern = pattern_ellipse(radius=size, plot=False)
    set_pattern(dsp, pattern)
    print("\rCircle radius: {0:d}".format(int(size)), end="       ")
    time.sleep(1)

In [ ]:
shutdown(dsp)